In [98]:
import pandas as pd
import yfinance as yf
import pandas_ta as ta

In [99]:
# Sample Inputs
total_investment = 1000  # Total investment amount
start_date = "2024-05-01"  # Investment period start date
end_date = "2024-05-30"  # Investment period end date
end_date2 = "2024-05-28"  # Investment period end date

In [100]:
stocks = [
	{"Ticker": "ADANIPORTS.NS", "Weightage": 35.0000},
	{"Ticker": "APOLLOHOSP.NS", "Weightage": 30.0000},
	{"Ticker": "ASIANPAINT.NS", "Weightage": 35.0000},]


In [101]:
def calculate_rsi(data, periods=14):
	"""
	Calculates the Relative Strength Index (RSI) for a given DataFrame.

	Args:
		data: pandas DataFrame containing the 'Close' price data.
		periods: Number of periods for RSI calculation (default: 14).

	Returns:
		pandas Series containing the RSI values.
	"""

	delta = data["Close"].diff()
	gain = (delta.where(delta > 0, 0)).fillna(0)
	loss = (delta.where(delta < 0, 0)).fillna(0)

	avg_gain = gain.rolling(window=periods).mean()
	avg_loss = abs(loss.rolling(window=periods).mean())

	rs = avg_gain / avg_loss
	rsi = 100 - (100 / (1 + rs))

	print(rsi)

	return rsi

In [102]:
# Convert dates to Timestamps
start_date = pd.Timestamp(start_date)
end_date = pd.Timestamp(end_date)


# Function to adjust weightage by RSI
def adjust_weightage_by_rsi(weightage, rsi):
    print(rsi)
    rsi = rsi.iloc[0] if isinstance(rsi, pd.Series) else rsi
    if rsi is None or pd.isna(rsi):
        return weightage  # If RSI is missing, return the original weightage

    if rsi > 90:
        return weightage * 0.7  # Reduce by 30%
    elif rsi > 80:
        return weightage * 0.8  # Reduce by 20%
    elif rsi > 70:
        return weightage * 0.9  # Reduce by 10%
    elif rsi < 10:
        return weightage * 1.3  # Increase by 30%
    elif rsi < 20:
        return weightage * 1.2  # Increase by 20%
    elif rsi < 30:
        return weightage * 1.1  # Increase by 10%
    return weightage  # No change if between 30 and 70


# Results list
results = []

# Loop through each stock
for stock in stocks:
    ticker = stock["Ticker"]
    weightage = stock["Weightage"]

    # Download stock data
    data = yf.download(ticker, start=start_date, end=end_date)
    if data.empty:
        print(f"Data for {ticker} is not available. Skipping.")
        continue

    # Calculate RSI
    data["RSI"] = calculate_rsi(data, periods=14)

    # Adjust weightage dynamically
    data["Adjusted Weightage"] = data["RSI"].apply(
        lambda x: adjust_weightage_by_rsi(weightage, x)
    )

    # Normalize adjusted weightage to ensure proportional allocation
    total_adjusted_weightage = data["Adjusted Weightage"].sum()
    data["Normalized Weightage"] = data["Adjusted Weightage"] / total_adjusted_weightage

    # Calculate allocated investment and shares
    data["Allocated Investment"] = data["Normalized Weightage"] * total_investment

    # Safely calculate shares
    data["Shares"] = data.apply(
        lambda row: (
            row["Allocated Investment"].iloc[0] // row["Close"].iloc[0]
            if row["Close"].iloc[0] > 0
            else 0
        ),
        axis=1,
    )

    # Append results
    for index, row in data.iterrows():
        results.append(
            {
                "Ticker": ticker,
                "Date": index.date(),
                "Weightage": weightage,
                "Adjusted Weightage": row["Adjusted Weightage"].iloc[0],
                "Normalized Weightage": row["Normalized Weightage"].iloc[0],
                "Allocated Investment": row["Allocated Investment"].iloc[0],
                "Open Price": row["Open"].iloc[0],  # Include Open Price
                "Close Price": row["Close"].iloc[0],
                "RSI": row["RSI"].iloc[0],
                "Shares": row["Shares"].iloc[0],
            }
        )

# Convert results to DataFrame
summary = pd.DataFrame(results)

# Reorder and display columns
summary = summary[
    [
        "Ticker",
        "Date",
        "Weightage",
        "Adjusted Weightage",
        "Normalized Weightage",
        "Allocated Investment",
        "Open Price",
        "Close Price",
        "RSI",
        "Shares",
    ]
]

# Save the summary to CSV
summary.to_csv("investment_summary.csv", index=False)
print("\nSummary saved to 'investment_summary.csv'")

[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Ticker      ADANIPORTS.NS
Date                     
2024-05-02            NaN
2024-05-03            NaN
2024-05-06            NaN
2024-05-07            NaN
2024-05-08            NaN
2024-05-09            NaN
2024-05-10            NaN
2024-05-13            NaN
2024-05-14            NaN
2024-05-15            NaN
2024-05-16            NaN
2024-05-17            NaN
2024-05-21            NaN
2024-05-22      57.542089
2024-05-23      66.018057
2024-05-24      64.294220
2024-05-27      72.119887
2024-05-28      66.160660
2024-05-29      69.059570
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
57.54208898975209
66.01805674367421
64.29422022402494
72.11988723651329
66.16066000079014
69.05957027073828
Ticker      APOLLOHOSP.NS
Date                     
2024-05-02            NaN
2024-05-03            NaN
2024-05-06            NaN
2024-05-07            NaN
2024-05-08            NaN
2024-05-09            NaN
2024-05-10            NaN
2024-05-13            NaN
2024-05-14            NaN
2024-05-